# Day 4: Passport Processing

You arrive at the airport only to realize that you grabbed your North Pole Credentials instead of your passport. While these documents are extremely similar, North Pole Credentials aren't issued by a country and therefore aren't actually valid documentation for travel in most of the world.

It seems like you're not the only one having problems, though; a very long line has formed for the automatic passport scanners, and the delay could upset your travel itinerary.

Due to some questionable network security, you realize you might be able to solve both of these problems at the same time.

The automatic passport scanners are slow because they're having trouble detecting which passports have all required fields. The expected fields are as follows:

```text
    byr (Birth Year)
    iyr (Issue Year)
    eyr (Expiration Year)
    hgt (Height)
    hcl (Hair Color)
    ecl (Eye Color)
    pid (Passport ID)
    cid (Country ID)
```

Passport data is validated in batch files (your puzzle input). Each passport is represented as a sequence of key:value pairs separated by spaces or newlines. Passports are separated by blank lines.

Here is an example batch file containing four passports:

```text
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
```

The first passport is valid - all eight fields are present. The second passport is invalid - it is missing hgt (the Height field).

The third passport is interesting; the only missing field is cid, so it looks like data from North Pole Credentials, not a passport at all! Surely, nobody would mind if you made the system temporarily ignore missing cid fields. Treat this "passport" as valid.

The fourth passport is missing two fields, cid and byr. Missing cid is fine, but missing any other field is not, so this passport is invalid.

According to the above rules, your improved system would report 2 valid passports.

## Puzzle 1

Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

In [1]:
# Python imports
import re

from dataclasses import dataclass
from pathlib import Path
from typing import Optional

Here we take advantage of Python's `@dataclass` decorator. This allows us to create a class that does nothing (much) but store data in a particular format. We can specify the data type of each field in the object, and if a field is optional we can assign it a default value of `None`.

If we don't instantiate the class with the appropriate data in each case, an error is raised. So we can attempt to create a passport and, if there is missing data, an error will be raised. So, "invalid" passports will not be created. 

Our solution is to try to create a `Passport` object for each dataset, and count the successful creations - these are all valid passports.

In [2]:
@dataclass
class Passport:
    
    """Data class representing a generic passport"""
        
    byr: int
    iyr: int
    eyr: int
    hgt: int
    hcl: str
    ecl: str
    pid: int
    cid: int=None

In [3]:
def parse_passports(fpath):
    """Return a list of passwords from file fpath
    
    :param fpath:  path to batch file of passwords
    """
    passports = []
    
    with Path(fpath).open("r") as ifh:
        data = {}
        for line in ifh.readlines():
            lndata = line.strip().split()
            # If line is blank, create a new Passport
            if len(lndata) == 0:
                try:
                    passports.append(Passport(**data))
                except (TypeError, AssertionError, ValueError) as err:  # invalid passport
                    pass
                data = {}  # clear data
            else:
                kvs = [tuple(_.split(":")) for _ in lndata]
                for key, val in kvs:
                    data[key] = val
                    
    return passports

Solve the test puzzle:

In [4]:
passports = parse_passports("day04_test.txt")
len(passports)

2

Solve the real puzzle

In [5]:
passports = parse_passports("day04_data.txt")
len(passports)

237

## Puzzle 2

The line is moving more quickly now, but you overhear airport security talking about how passports with invalid data are getting through. Better add some data validation, quick!

You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

```text
    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.
```

Your job is to count the passports where all required fields are both present and valid according to the above rules. Here are some example values:

```text
byr valid:   2002
byr invalid: 2003

hgt valid:   60in
hgt valid:   190cm
hgt invalid: 190in
hgt invalid: 190

hcl valid:   #123abc
hcl invalid: #123abz
hcl invalid: 123abc

ecl valid:   brn
ecl invalid: wat

pid valid:   000000001
pid invalid: 0123456789
```

Here are some invalid passports:

```text
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
```

Here are some valid passports:

```text
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
```

Count the number of valid passports - those that have all required fields and valid values. Continue to treat cid as optional. In your batch file, how many passports are valid?

We reimplement a different solution for a `Passport` class, to use Python's `@property` decorator and `.setter()` methods. This requires quite a few changes.

- We can't rely on `@dataclass` to create boilerplate, we have to implement `__init__()`
- We write a getter and setter for each data field. The setter allows us to assert properties of the data (e.g. length and/or value) and raise an error (`TypeError`, `ValueError` or `AssertionError` usually) if this is incorrect.

As before, though, "invalid" details will not result in successful creation of a `Passport` object, so we can just count the number of successful passport creations to get our answer.

In [6]:
class Passport:
    
    """Data class representing a generic passport"""
        
    def __init__(self, byr: int=0, iyr: int=0, eyr: int=0, hgt: str="", 
                 hcl: str="", ecl: str="", pid: str="", cid: Optional[int]=0):
        """Instantiate Passport
        
        :param byr: int  # (Birth Year)
        :param iyr: int  # (Issue Year)
        :param eyr: int  # (Expiration Year)
        :param hgt: int  # (Height)
        :param hcl: str  # (Hair Color)
        :param ecl: str  # (Eye Color)
        :param pid: int  # (Passport ID)
        :param cid: int=None  # (Country ID)
        """
        self.byr = int(byr)
        self.iyr = int(iyr)
        self.eyr = int(eyr)
        self.hgt = str(hgt)
        self.hcl = str(hcl)
        self.ecl = str(ecl)
        self.pid = str(pid)
        self.cid = int(cid)
        
    @property
    def byr(self):
        """Birth year"""
        return self._byr
    
    @byr.setter
    def byr(self, val):
        """Birth year"""
        val = int(val)
        assert 1920 <= val <= 2002
        self._byr = val

    @property
    def iyr(self):
        """Issue year"""
        return self._iyr
    
    @iyr.setter
    def iyr(self, val):
        """Issue year"""
        val = int(val)
        assert 2010 <= val <= 2020
        self._iyr = val        

    @property
    def eyr(self):
        """Expiry year"""
        return self._eyr
    
    @eyr.setter
    def eyr(self, val):
        """Expiry year"""
        val = int(val)
        assert 2020 <= val <= 2030
        self._eyr = val        
        
    @property
    def hgt(self):
        """Height"""
        return self._hgt
    
    @hgt.setter
    def hgt(self, inval):
        """Birth year"""
        if inval.endswith("cm"):
            val = int(inval.split("cm")[0])
            assert 150 <= val <= 193
        elif inval.endswith("in"):
            val = int(inval.split("in")[0])
            assert 59 <= val <= 76
        else:
            raise TypeError("Height must be inches or cm.")
        self._hgt = inval
    
    @property
    def hcl(self):
        """Hair colour"""
        return self._hcl
    
    @hcl.setter
    def hcl(self, inval):
        """Hair colour"""
        val = re.match("#[0-9a-f]{6}$", str(inval))  # the $ asserts that the string stops
        if val is None:
            raise TypeError(f"Hair colour must be #[0-9a-f]{{6}}$, got {inval}")
        self._hcl = val.group()
        
    @property
    def ecl(self):
        """Eye colour"""
        return self._ecl
    
    @ecl.setter
    def ecl(self, val):
        """Eye colour"""
        assert val in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
        self._ecl = val
        
    @property
    def pid(self):
        """Passport ID"""
        return self._pid
    
    @pid.setter
    def pid(self, inval):
        """Passport ID"""
        val = re.match("[0-9]{9}$", str(inval))  # the $ asserts that the string stops
        if val is None:
            raise TypeError(f"Passport ID must be [0-9]{{9}}$, got {inval}")
        self._pid = val.group()


Solve the expanded set of test cases.

In [7]:
passports = parse_passports("day04_test.txt")
len(passports)

2

In [8]:
passports = parse_passports("day04_test02.txt")
len(passports)

0

In [9]:
passports = parse_passports("day04_test03.txt")
len(passports)

3

Solve the real problem.

In [10]:
passports = parse_passports("day04_data.txt")
len(passports)

172